In [1]:
import requests
import pandas as pd
import functions
from importlib import reload
reload(functions)

<module 'functions' from '/mnt/c/DEV/Masterarbeit/code/functions.py'>

In [2]:
package_name = "torch"
response = requests.get(f"https://pypi.org/pypi/{package_name}/json")
pypi_data = response.json()

In [3]:
owner, repo, repo_link = functions.get_pypi_repo(pypi_data)
repo_link

'https://github.com/pytorch/pytorch'

In [4]:
pd.set_option('display.max_rows', None)

## Git quick stats

In [5]:
git_contributors_df = await functions.get_git_contributors(owner, repo, repo_link, package_name)
git_contributors_df.head(10)

,name,email,insertions,deletions,lines_changed,files,commits,first_commit,last_commit
0,PyTorch MergeBot,pytorchmergebot@users.noreply.github.com,466516,483184,949700,18807,2497,2022-01-26 23:11:23+00:00,2024-09-11 21:27:53+00:00
1,Edward YangEdward Z. Yang,ezyang@fb.com,187930,179156,367086,12242,1925,2017-04-18 16:37:21-04:00,2023-09-12 03:44:24+00:00
2,zou3519rzouRichard Zourichard,zou3519@gmail.com,128565,48898,177463,4617,1324,2017-10-09 08:42:23-07:00,2024-09-10 08:00:03-07:00
3,Nikita Shulga,nshulga@fb.com,174308,151681,325989,7477,1202,2019-12-20 09:10:47-08:00,2023-10-03 08:16:42+00:00
4,Edward Z. YangEdward Yang,ezyang@meta.com,79482,38754,118236,4246,1007,2022-09-30 03:19:09+00:00,2024-09-11 07:54:34-07:00
5,Jerry Zhang,jerryzh@fb.com,110881,85718,196599,6105,970,2017-08-17 09:16:53-07:00,2023-11-29 08:58:24+00:00
6,Peter Bellpeterbell10,peterbell10@live.co.uk,109713,71838,181551,5461,890,2019-09-10 09:17:09-07:00,2024-07-31 13:16:11+01:00
7,Adam Paszke,adam.paszke@gmail.com,123938,57140,181078,5428,870,2015-11-11 19:19:43+01:00,2020-03-09 10:21:50-07:00
8,Gregory Chanan,gchanan@fb.com,54565,50579,105144,3515,855,2016-08-31 12:23:51-07:00,2021-05-13 08:09:53-07:00
9,Michael Suosuo,suo@fb.com,157309,142386,299695,5626,791,2018-08-01 19:06:19-07:00,2024-03-05 13:37:45-08:00


## PyPi Owner (verified)

PyPi Owner sind in PyPi immer Organisationen, deswegen brauche ich diese nicht zu verarbeiten.

## PyPi Maintainers (verified)

In [6]:
pypi_maintainers_df = await functions.get_pypi_maintainers(package_name)
pypi_maintainers_df

,login,name
0,atalman,Andrey Talman
1,facebook,facebook
2,malfet,Nikita
3,seemethere,Eli Uriegas
4,soumith,None


Kann keine logins matchen, da ich die aus git nicht bekomme. Aus dem Grund matche ich jetzt logins mit emails.

In [7]:
result = functions.matching(pypi_maintainers_df, git_contributors_df)
result

,login,name,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
4,soumith,None,22,16750,8479,25229,1022,462,2012-08-23 19:03:40-04:00,2024-07-20 16:02:42+00:00,0.5
0,atalman,Andrey Talman,79,32666,52660,85326,802,218,2021-11-05 12:48:37-07:00,2024-09-10 23:46:36+00:00,1.0
2,malfet,Nikita,118,609,1007,1616,178,159,2023-09-27 18:49:03+00:00,2024-09-11 00:20:53+00:00,1.0
1,facebook,facebook,130,141,141,282,142,142,2017-09-29 16:29:31-07:00,2023-12-06 22:47:34+00:00,1.0
3,seemethere,Eli Uriegas,1027,226,63,289,17,6,2020-05-21 13:05:21-07:00,2022-10-28 15:12:31-07:00,1.0


## Python Authors (not verified)

Assuming Author seperation by ,

In [8]:
python_authors_df = functions.get_python_authors(pypi_data)
python_authors_df

,name,email
0,PyTorch Team,packages@pytorch.org


In [9]:
result = functions.matching(python_authors_df, git_contributors_df)
result

,name,email,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
0,PyTorch Team,packages@pytorch.org,2530,3,1,4,1,1,2023-06-27 18:41:21+00:00,2023-06-27 18:41:21+00:00,0.5


## Python Maintainers (not verified)

In [10]:
python_maintainers_df = functions.get_python_maintainers(pypi_data)
python_maintainers_df

""


In [11]:
result = functions.matching(python_maintainers_df, git_contributors_df)
result

,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score


## Citation File Format (Authors)

In [12]:
cff_df = functions.get_cff_authors(owner, repo)
cff_df

,name,email,ORCID,type,date-released
0,PyTorch Team,None,None,None,None


In [13]:
result = functions.matching(cff_df, git_contributors_df)
result

,name,email,ORCID,type,date-released,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
0,PyTorch Team,None,None,None,None,2530,3,1,4,1,1,2023-06-27 18:41:21+00:00,2023-06-27 18:41:21+00:00,0.5


## Citation File Format (Preferred citation Authors)

In [14]:
cff_df = functions.get_cff_preferred_citation_authors(owner, repo)
cff_df

,name,email,ORCID,type,year,date-published
0,Jason Ansel,None,None,conference-paper,2024,None
1,Edward Yang,None,None,conference-paper,2024,None
2,Horace He,None,None,conference-paper,2024,None
3,Natalia Gimelshein,None,None,conference-paper,2024,None
4,Animesh Jain,None,None,conference-paper,2024,None
5,Michael Voznesensky,None,None,conference-paper,2024,None
6,Bin Bao,None,None,conference-paper,2024,None
7,Peter Bell,None,None,conference-paper,2024,None
8,David Berard,None,None,conference-paper,2024,None
9,Evgeni Burovski,None,None,conference-paper,2024,None


In [15]:
result = functions.matching(cff_df, git_contributors_df)
result

,name,email,ORCID,type,year,date-published,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
1,Edward Yang,None,None,conference-paper,2024,None,2.0,187930.0,179156.0,367086.0,12242.0,1925.0,2017-04-18 16:37:21-04:00,2023-09-12 03:44:24+00:00,0.5
44,Richard Zou,None,None,conference-paper,2024,None,3.0,128565.0,48898.0,177463.0,4617.0,1324.0,2017-10-09 08:42:23-07:00,2024-09-10 08:00:03-07:00,0.5
7,Peter Bell,None,None,conference-paper,2024,None,7.0,109713.0,71838.0,181551.0,5461.0,890.0,2019-09-10 09:17:09-07:00,2024-07-31 13:16:11+01:00,0.5
46,Gregory Chanan,None,None,conference-paper,2024,None,9.0,54565.0,50579.0,105144.0,3515.0,855.0,2016-08-31 12:23:51-07:00,2021-05-13 08:09:53-07:00,0.5
36,Michael Suo,None,None,conference-paper,2024,None,10.0,157309.0,142386.0,299695.0,5626.0,791.0,2018-08-01 19:06:19-07:00,2024-03-05 13:37:45-08:00,0.5
14,Zachary DeVito,None,None,conference-paper,2024,None,14.0,112240.0,73839.0,186079.0,4886.0,565.0,2017-06-20 16:49:13-07:00,2022-12-09 16:44:31-08:00,0.5
4,Animesh Jain,None,None,conference-paper,2024,None,17.0,32427.0,10039.0,42466.0,1770.0,518.0,2021-10-11 12:46:14-07:00,2024-09-11 14:18:34-07:00,0.5
21,Kshiteej Kalambarkar,None,None,conference-paper,2024,None,18.0,44689.0,23162.0,67851.0,2014.0,506.0,2019-03-21 13:10:34-07:00,2023-11-21 18:55:51+00:00,0.5
48,Soumith Chintala,None,None,conference-paper,2024,None,22.0,16750.0,8479.0,25229.0,1022.0,462.0,2012-08-23 19:03:40-04:00,2024-07-20 16:02:42+00:00,0.5
15,Elias Ellison,None,None,conference-paper,2024,None,30.0,41315.0,19752.0,61067.0,1805.0,394.0,2018-07-30 15:43:29-07:00,2022-06-17 02:56:06+00:00,0.5


## Citation.bib

In [16]:
bib_df = functions.get_bib_authors(owner, repo)
bib_df

""


In [17]:
result = functions.matching(bib_df, git_contributors_df)
result

,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score


## Description

In [18]:
description = pypi_data['info']['description']
description_df = functions.get_description_authors(description)
description_df

,name
0,Soumith Chintala](http://soumith.ch
1,Gregory Chanan](https://github.com
2,Dmytro Dzhulgakov](https://github.com
3,Edward Yang](https://github.com
4,Nikita Shulga](https://github.com
5,Trevor Killeen
6,Sasank Chilamkurthy
7,Sergey Zagoruyko
8,Adam Lerer
9,Francisco Massa


In [19]:
result = functions.matching(description_df, git_contributors_df)
result

,name,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
22,Edward Yang,2.0,187930.0,179156.0,367086.0,12242.0,1925.0,2017-04-18 16:37:21-04:00,2023-09-12 03:44:24+00:00,1.0
4,Nikita Shulga](https://github.com,4.0,174308.0,151681.0,325989.0,7477.0,1202.0,2019-12-20 09:10:47-08:00,2023-10-03 08:16:42+00:00,1.0
1,Gregory Chanan](https://github.com,9.0,54565.0,50579.0,105144.0,3515.0,855.0,2016-08-31 12:23:51-07:00,2021-05-13 08:09:53-07:00,1.0
23,Zachary Devito,14.0,112240.0,73839.0,186079.0,4886.0,565.0,2017-06-20 16:49:13-07:00,2022-12-09 16:44:31-08:00,1.0
19,Natalia Gimelshein,34.0,22333.0,30777.0,53110.0,1447.0,386.0,2019-09-25 17:34:50-07:00,2023-06-03 01:45:09+00:00,1.0
12,Alban Desmaison,86.0,11631.0,21948.0,33579.0,798.0,203.0,2019-10-30 13:54:24-07:00,2024-07-31 23:50:00+00:00,1.0
2,Dmytro Dzhulgakov](https://github.com,91.0,11533.0,5948.0,17481.0,766.0,192.0,2016-11-23 18:17:01-08:00,2022-02-11 14:43:45-08:00,1.0
5,Trevor Killeen,119.0,8008.0,4598.0,12606.0,453.0,155.0,2016-09-28 15:03:09-07:00,2017-08-30 12:20:13-07:00,1.0
13,Andreas Koepf,228.0,19801.0,8444.0,28245.0,319.0,70.0,2015-12-11 23:47:10+01:00,2020-01-07 10:02:23-08:00,1.0
18,Marat Dukhan,236.0,7328.0,2555.0,9883.0,244.0,66.0,2017-07-03 22:14:27-07:00,2019-01-17 09:18:04-08:00,1.0


## README.md

In [20]:
readme_df = functions.get_readme_authors(owner, repo)
readme_df

,name
0,Soumith Chintala](http://soumith.ch
1,Gregory Chanan](https://github.com
2,Dmytro Dzhulgakov](https://github.com
3,Edward Yang](https://github.com
4,Nikita Shulga](https://github.com
5,Trevor Killeen](https://github.com
6,Sasank Chilamkurthy](https://github.com
7,Sergey Zagoruyko](https://github.com
8,Adam Lerer](https://github.com
9,Francisco Massa](https://github.com


In [21]:
result = functions.matching(readme_df, git_contributors_df)
result

,name,rank,insertions,deletions,lines_changed,files,commits,first_commit,last_commit,score
4,Nikita Shulga](https://github.com,4.0,174308.0,151681.0,325989.0,7477.0,1202.0,2019-12-20 09:10:47-08:00,2023-10-03 08:16:42+00:00,1.0
1,Gregory Chanan](https://github.com,9.0,54565.0,50579.0,105144.0,3515.0,855.0,2016-08-31 12:23:51-07:00,2021-05-13 08:09:53-07:00,1.0
19,Natalia Gimelshein](https://github.com,34.0,22333.0,30777.0,53110.0,1447.0,386.0,2019-09-25 17:34:50-07:00,2023-06-03 01:45:09+00:00,1.0
2,Dmytro Dzhulgakov](https://github.com,91.0,11533.0,5948.0,17481.0,766.0,192.0,2016-11-23 18:17:01-08:00,2022-02-11 14:43:45-08:00,1.0
5,Trevor Killeen](https://github.com,119.0,8008.0,4598.0,12606.0,453.0,155.0,2016-09-28 15:03:09-07:00,2017-08-30 12:20:13-07:00,1.0
22,Zachary Devito](https://github.com,186.0,17707.0,10870.0,28577.0,814.0,93.0,2017-07-10 07:42:45-07:00,2023-06-16 06:32:18+00:00,1.0
18,Marat Dukhan](https://github.com,236.0,7328.0,2555.0,9883.0,244.0,66.0,2017-07-03 22:14:27-07:00,2019-01-17 09:18:04-08:00,1.0
11,Luca Antiga](https://github.com,289.0,6284.0,1521.0,7805.0,238.0,53.0,2016-10-16 17:49:47+02:00,2020-05-28 01:26:44-07:00,1.0
20,Christian Sarofeen](https://github.com,295.0,48616.0,17306.0,65922.0,492.0,51.0,2017-02-17 10:16:12-08:00,2020-08-18 09:07:55-07:00,1.0
10,Alykhan Tejani](https://github.com,326.0,1735.0,281.0,2016.0,85.0,41.0,2017-01-21 15:41:52+00:00,2018-01-02 17:56:49+00:00,1.0
